In [3]:
!pip install tqdm

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


In [4]:
!pip install tenacity

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


In [1]:
import os
import io
import json, glob
import urllib3
from PIL import *
import PIL.Image
import numpy as np
import pandas as pd
from tenacity import *

In [2]:
def parse_dataset(dataset, outdir, _max=10000):
    """
    parse the dataset to create a list of tuple containing absolute path and url of image
    :param _dataset: dataset to parse
    :param _outdir: output directory where data will be saved
    :param _max: maximum images to download (change to download all dataset)
    :return: list of tuple containing absolute path and url of image
    """
    _fnames_urls = []
    with open(dataset, 'r') as f:
        data = json.load(f)
        for image in data["images"]:
            url = image["url"]
            fname = os.path.join(outdir, "{}.jpg".format(image["imageId"]))
            _fnames_urls.append((fname, url))
    return _fnames_urls[:_max]
def download_image(fnames_and_urls):
    """
    download image and save its with 90% quality as JPG format
    skip image downloading if image already exists at given path
    :param fnames_and_urls: tuple containing absolute path and url of image
    """
    fname, url = fnames_and_urls
    if not os.path.exists(fname):
        http = urllib3.PoolManager(retries=Retry(connect=3, read=2, redirect=3))
        response = http.request("GET", url)
        image = Image.open(io.BytesIO(response.data))
        image_rgb = image.convert("RGB")
        image_rgb.save(fname, format='JPEG', quality=90)
        
%cd '/home/ec2-user/SageMaker/imat/'

parse = parse_dataset('validation.json', '/home/ec2-user/SageMaker/imat/validation_images',100)

for i in parse:
    download_image(i)

/home/ec2-user/SageMaker/imat


In [3]:
val_json = json.load(open('/home/ec2-user/SageMaker/imat/validation.json'))

In [4]:
num_images = len(val_json['images'])
print(num_images)

9897


In [5]:
# Get image ids and labels
image_id = []
label_id = []
for i in range(num_images):
    image_id.append(int(val_json['annotations'][i]['imageId']))
    label_list = val_json['annotations'][i]['labelId']
    label_str = ''
    for j in range(len(label_list)):
        label_str += ' ' + label_list[j]
    label_str = label_str[1:]
    label_id.append(label_str)

In [7]:
len(label_id)

9897

In [14]:
def get_index(path):
    return int(path.split('/')[-1].split('.')[0])


In [30]:
# Get validation image paths
validation_path_format = '/home/ec2-user/SageMaker/imat/validation_images/*.jpg'
image_path = sorted(glob.glob(validation_path_format), key = get_index)

TypeError: get_index() takes exactly 1 argument (2 given)

In [29]:
len(image_path)

0

In [149]:
validation_csv = pd.DataFrame({'imageId': image_id, 'labelId': label_id, 'imagePath': image_path})

In [172]:
validation_csv.head()

,imageId,imagePath,labelId
0,1,/home/ec2-user/SageMaker/imat/validation_image...,62 17 66 214 105 137 85
1,2,/home/ec2-user/SageMaker/imat/validation_image...,95 17 66 214 164 137 20 204 184
2,3,/home/ec2-user/SageMaker/imat/validation_image...,122 19 66 186 180 44 154 20
3,4,/home/ec2-user/SageMaker/imat/validation_image...,190 222 66 153 164 226 53 184
4,5,/home/ec2-user/SageMaker/imat/validation_image...,62 66 153 171 111 137 70 204 184
